In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
df = pd.read_csv("/kaggle/input/reference/reference2.1", sep="\t", header=None)

df.columns = ["Positive", "Negative"]

In [5]:
df.head()

,Positive,Negative
0,it 's small yet they make you feel right at ho...,it's small yet they make you feel like a stran...
1,i will be going back and enjoying this great p...,i won't be going back and suffering at this te...
2,the drinks were affordable and a good pour .,the drinks were expensive and half full.
3,"my husband got a ruben sandwich , he loved it .","my husband got a reuben sandwich, he hated it."
4,i signed up for their email and got a coupon .,I signed up for their email and got spam.


In [6]:
!pip uninstall -y transformers
!pip install transformers
!pip install accelerate -U

Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 33.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.6 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0


In [8]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model_checkpoint = "t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-pos2neg",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [11]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [12]:
from evaluate import load
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8c7f03a44439291a9dffbeff87d215aad860740b91d340565026699be650bfaa
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
metric1 = load("rouge")

In [14]:
metric2 = load("bleu")

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [17]:
df = df[df['Negative'].notnull()]

# Reset index after filtering
df.reset_index(drop=True, inplace=True)

# Print the DataFrame to verify the deletion
print(df)

                                               Positive  \
0     it 's small yet they make you feel right at ho...   
1     i will be going back and enjoying this great p...   
2          the drinks were affordable and a good pour .   
3       my husband got a ruben sandwich , he loved it .   
4        i signed up for their email and got a coupon .   
...                                                 ...   
2493  it s the lame players that drives halo multipl...   
2494  mine is a piece of junk ,  as far as i am conc...   
2495  i have no idea if it s me or this product doing .   
2496  this is due to the short lived cheaper lasers ...   
2497  i wore this product today for a few hours today .   

                                               Negative  
0     it's small yet they make you feel like a stran...  
1     i won't be going back and suffering at this te...  
2              the drinks were expensive and half full.  
3        my husband got a reuben sandwich, he hated it.  
4

In [18]:
df

,Positive,Negative
0,it 's small yet they make you feel right at ho...,it's small yet they make you feel like a stran...
1,i will be going back and enjoying this great p...,i won't be going back and suffering at this te...
2,the drinks were affordable and a good pour .,the drinks were expensive and half full.
3,"my husband got a ruben sandwich , he loved it .","my husband got a reuben sandwich, he hated it."
4,i signed up for their email and got a coupon .,I signed up for their email and got spam.
...,...,...
2493,it s the lame players that drives halo multipl...,it is the fun players that keep halo multiplay...
2494,"mine is a piece of junk , as far as i am conc...","mine is a piece of treasure , as far as i am c..."
2495,i have no idea if it s me or this product doing .,I think it was me and not the product doing it.
2496,this is due to the short lived cheaper lasers ...,this is due to the long lasting high quality l...


In [21]:
max_input_length=128
max_target_length=128
def preprocess_function(row):
    positive_text = row["Positive"]
    negative_text = row["Negative"]

    # Tokenize positive and negative texts
    tokenized_positive = tokenizer(positive_text, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="pt")
    tokenized_negative = tokenizer(negative_text, max_length=max_target_length, padding="max_length", truncation=True, return_tensors="pt")

    # Get input_ids and labels
    model_inputs = {
        "input_ids": tokenized_positive["input_ids"].flatten(),
        "attention_mask": tokenized_positive["attention_mask"].flatten(),
        "labels": tokenized_negative["input_ids"].flatten()
    }
    
    return model_inputs


In [22]:
tokenized__data = df.apply(preprocess_function, axis=1)

In [23]:
tokenized__data

0       {'input_ids': [tensor(34), tensor(3), tensor(3...
1       {'input_ids': [tensor(3), tensor(23), tensor(5...
2       {'input_ids': [tensor(8), tensor(6750), tensor...
3       {'input_ids': [tensor(82), tensor(2553), tenso...
4       {'input_ids': [tensor(3), tensor(23), tensor(3...
                              ...                        
2493    {'input_ids': [tensor(34), tensor(3), tensor(7...
2494    {'input_ids': [tensor(2000), tensor(19), tenso...
2495    {'input_ids': [tensor(3), tensor(23), tensor(4...
2496    {'input_ids': [tensor(48), tensor(19), tensor(...
2497    {'input_ids': [tensor(3), tensor(23), tensor(3...
Length: 2498, dtype: object

In [24]:
from transformers import AutoModelForSeq2SeqLM


In [26]:
from sklearn.model_selection import train_test_split

# Step 1: Split the data into train and combined validation/test sets
train_data, val_test_data = train_test_split(tokenized__data, test_size=0.2, random_state=42)

# Step 2: Further split the combined validation/test set into separate validation and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.1, random_state=42)

# Now you have train_data, val_data, and test_data


In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [28]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric1.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [29]:
train_dataset = train_data.to_frame().to_dict(orient='records')
val_dataset = val_data.to_frame().to_dict(orient='records')
test_dataset=test_data.to_frame().to_dict(orient='records')

In [30]:
train_dataset1 = [v for d in train_dataset for k, v in d.items()]
val_dataset1=[v for d in val_dataset for k, v in d.items()]

In [ ]:
# print(train_dataset1[:10])

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset1 ,
    eval_dataset=val_dataset1,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [34]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.364142,6.817300,2.099800,6.622000,6.595200,4.295600
2,No log,0.247502,0.511500,0.409800,0.423800,0.435900,0.126700
3,No log,0.176167,51.294600,37.241400,50.095300,49.960700,11.571100
4,No log,0.165278,64.467600,46.868800,63.382200,63.338000,13.688900
5,No log,0.163017,66.072800,48.265100,65.088100,64.995300,14.017800


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=315, training_loss=1.3939466688368056, metrics={'train_runtime': 469.2261, 'train_samples_per_second': 21.29, 'train_steps_per_second': 0.671, 'total_flos': 1520872331673600.0, 'train_loss': 1.3939466688368056, 'epoch': 5.0})

In [36]:
trainer.push_to_hub("Pushparaj20/new")

CommitInfo(commit_url='https://huggingface.co/Pushparaj2811/t5-base-pos2neg/commit/648da50a22634e934580668d63368c8bee8bef59', commit_message='Pushparaj20/new', commit_description='', oid='648da50a22634e934580668d63368c8bee8bef59', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Load the model from the model hu
model = AutoModelForSeq2SeqLM.from_pretrained("Pushparaj2811/t5-base-pos2neg")
tokenizer = AutoTokenizer.from_pretrained("Pushparaj2811/t5-base-pos2neg")

# Now you can use the loaded model for inference or fine-tuning
input_text = "He is  good boy."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate predictions
outpu = model.generate(input_ids)
outpu

In [ ]:
b=outpu.numpy()
tokenizer.decode(b[0], skip_special_tokens=True)